# Part 1

In [6]:
data = open("input.txt").read()[:-1].split("\n")
print(data[:5])

['R 2', 'D 2', 'U 2', 'D 1', 'L 1']


In [7]:
class Board:
    def __init__(self):
        self.board = [['H']]
        self.H = [0,0]
        self.T = [0,0]
        self.dim = [1,1]
        self.H_on_seen = False
    
    def __str__(self):
        return "\n".join(["".join(row) for row in self.board])

    def perform(self, cmd):
        direction, distance = cmd.split()
        distance = int(distance)
        match(direction):
            case "R":
                for _ in range(distance):
                    self.move((0,1))
            case "L":
                for _ in range(distance):
                    self.move((0,-1))
            case "U":
                for _ in range(distance):
                    self.move((-1,0))
            case "D":
                for _ in range(distance):
                    self.move((1,0))
    
    def process(self, commands):
        for command in commands:
            self.perform(command)
    
    def move(self, ofst):
        new_H = [self.H[0]+ofst[0], self.H[1]+ofst[1]]
        
        if new_H[0] == -1:
            self.expand_up()
            new_H[0] = 0
            
        elif new_H[0] == self.dim[0]:
            self.expand_down()

        elif new_H[1] == -1:
            self.expand_left()
            new_H[1] = 0

        elif new_H[1] == self.dim[1]:
            self.expand_right()
        
        if 2 in [abs(new_H[i]-self.T[i]) for i in [0,1]]:
            self.board[self.T[0]][self.T[1]] = '#'
            self.board[self.H[0]][self.H[1]] = 'T'
            self.T[0] = self.H[0]
            self.T[1] = self.H[1]
        else:
            if self.H_on_seen:
                self.board[self.H[0]][self.H[1]] = '#'
            else:
                self.board[self.H[0]][self.H[1]] = '.'
        
        self.H[0] = new_H[0]
        self.H[1] = new_H[1]
        if self.board[self.H[0]][self.H[1]] in ['#','T']:
            self.H_on_seen = True
        else:
            self.H_on_seen = False
        self.board[self.H[0]][self.H[1]] = 'H'
    
    def expand_up(self):
        self.board.insert(0,['.']*self.dim[1])
        self.dim[0] += 1
        self.T[0] += 1
        self.H[0] += 1
        
    def expand_down(self):
        self.board.append(['.']*self.dim[1])
        self.dim[0] += 1

    def expand_left(self):
        for row in self.board:
            row.insert(0,'.')
        self.dim[1] += 1
        self.T[1] += 1
        self.H[1] += 1
    
    def expand_right(self):
        for row in self.board:
            row.append('.')
        self.dim[1] += 1
    
    def count(self):
        return sum([row.count('#') for row in self.board]) + self.H_on_seen + (self.H != self.T)

In [8]:
board = Board()
board.process(data)
print(board.count())

6243


# Part 2

In [9]:
# remade this since i had a better idea for implementation
# UPDATE: holy shit this is so much better
class Rope:
    def __init__(self, length):
        self.visited = {(0,0)}
        # self.knots = [[0,0]]*length
        self.knots = [[0,0] for _ in range(length)] # god i hate how array pointers work sometimes
        self.dims = [0]*4
    
    def __len__(self):
        return len(self.knots)
    
    def count(self):
        return len(self.visited)
    
    def perform(self, cmd):
        direction, distance = cmd.split()
        distance = int(distance)
        match(direction):
            case "R":
                for _ in range(distance):
                    self.__move((0,1))
            case "L":
                for _ in range(distance):
                    self.__move((0,-1))
            case "U":
                for _ in range(distance):
                    self.__move((-1,0))
            case "D":
                for _ in range(distance):
                    self.__move((1,0))
    
    def process(self, commands):
        for command in commands:
            # print('\t', command)
            self.perform(command)
    
    def __str__(self):
        board = ''
        for i in range(self.dims[2], self.dims[1]+1):
            row = ''
            for j in range(self.dims[0], self.dims[3]+1):
                if [i,j] in self.knots:
                    row += str(self.knots.index([i,j]))
                elif (i,j) in self.visited:
                    row += '#'
                else:
                    row += '.'
            board += row + '\n'
        return board
    
    def __move(self, ofst):
        self.knots[0][0] += ofst[0]
        self.knots[0][1] += ofst[1]
        
        if self.knots[0][0] < self.dims[2]:
            self.dims[2] = self.knots[0][0]
        elif self.knots[0][0] > self.dims[1]:
            self.dims[1] = self.knots[0][0]
        elif self.knots[0][1] < self.dims[0]:
            self.dims[0] = self.knots[0][1]
        elif self.knots[0][1] > self.dims[3]:
            self.dims[3] = self.knots[0][1]
        
        for i in range(1, len(self)):
            displacement = [self.knots[i-1][0] - self.knots[i][0], self.knots[i-1][1] - self.knots[i][1]]
            schmove = False # this can prob be done better but im tired
            for axis in [0,1]:
                if abs(displacement[axis]) == 2:
                    displacement[axis] //= 2
                    schmove = True
            if schmove:
                self.knots[i][0] += displacement[0]
                self.knots[i][1] += displacement[1]
                    
        # print(self)
        self.visited.add(tuple(self.knots[-1]))

In [11]:
rope = Rope(10)
rope.process(data)
print(rope.count())
# print(rope)

2630
